In [1]:
from zoo.common.nncontext import *
sc = init_nncontext("Callback Example")

In [2]:
class PythonPrinter(object):

    def __init__(self):
        pass

    def print(self, message):
        print(message)

    class Java:
        implements = ["com.intel.analytics.zoo.PythonPrinter"]

In [5]:
from py4j.java_gateway import java_import, JavaGateway, JavaObject, GatewayParameters

In [6]:
def ensure_callback_server_started(gw):
    """
    Start callback server if not already started. The callback server is needed if the Java
    driver process needs to callback into the Python driver process to execute Python code.
    """

    # getattr will fallback to JVM, so we cannot test by hasattr()
    if "_callback_server" not in gw.__dict__ or gw._callback_server is None:
        gw.callback_server_parameters.eager_load = True
        gw.callback_server_parameters.daemonize = True
        gw.callback_server_parameters.daemonize_connections = True
        gw.callback_server_parameters.port = 0
        gw.start_callback_server(gw.callback_server_parameters)
        cbport = gw._callback_server.server_socket.getsockname()[1]
        gw._callback_server.port = cbport
        # gateway with real port
        gw._python_proxy_port = gw._callback_server.port
        # get the GatewayServer object in JVM by ID
        jgws = JavaObject("GATEWAY_SERVER", gw._gateway_client)
        # update the port of CallbackClient with real port
        jgws.resetCallbackClient(jgws.getCallbackClient().getAddress(), gw._python_proxy_port)

In [7]:
gw = SparkContext._gateway
ensure_callback_server_started(gw)

In [9]:
pp = PythonPrinter()

In [10]:
for jinvoker in JavaCreator.instance("float", gw).value:
    try:
        api = getattr(jinvoker, "callBackToPython")
        api(pp)
    except Exception as e:
        error = e
        if "does not exist" not in str(e):
            raise e

printing in python iteration 0
printing in python iteration 1
printing in python iteration 2
printing in python iteration 3
printing in python iteration 4
printing in python iteration 5
printing in python iteration 6
printing in python iteration 7
printing in python iteration 8
printing in python iteration 9
